Part 1: Cleaning the Data / Preprocessing

Loading the Data

In [200]:
import pandas as pd
import numpy as np
import tabulate
import plotly.express as px

In [154]:
#Important Transformations: Excel 
# Remove Duplicates
# Remove any with start before 2019
# Define group by subs list from Rachel
# 

df = pd.read_csv("test_model_master_cleaned.csv", parse_dates=['start'])
df_letters = pd.read_csv("newsletters.csv")
df_alerts = pd.read_csv("alerts.csv")

/var/folders/3c/s8p5wvy902gfgnq64pms18vm0000gn/T/ipykernel_1497/2065892052.py:7: DtypeWarning: Columns (14,28,29,75,77,97) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("test_model_master_cleaned.csv", parse_dates=['start'])
/var/folders/3c/s8p5wvy902gfgnq64pms18vm0000gn/T/ipykernel_1497/2065892052.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_letters = pd.read_csv("newsletters.csv")


Recoding

In [155]:
#Creating Time Variables
df['sub_year'] = df.start.dt.year
df['sub_month'] = df.start.dt.month
df['sub_hour'] = df.start.dt.hour

In [156]:
#Dummies for sub type
df.loc[ df['Resource_name'] == "FP Digital Access", 'Resource_name'] = "Digital"
df = df.query("Resource_name == 'Premium' | Resource_name == 'Insider' | Resource_name == 'Digital'")
df = df.join(pd.get_dummies(df['Resource_name'], prefix="type"))

In [157]:
# dummies for acquisition link/source: article, landing page, other
# subscribe
# academic
# article
# other
# print_archive
df = df.join(pd.get_dummies(df['conversion'], prefix="conversion"))


Defining Variables

In [185]:
xvars = ['days_subscribed',
    'auto_renew',
    'logins_last_30',
    'pageviews_last_30',
    'promcode',
    'num_newsletters',
    'login_app',
    'sub_year',
    'sub_month',
    'type_Digital', 
    'type_Insider',
    'type_Premium',
    'covid',
    'ukraine',
    "sessions_last_30",
    'group',
    'commented',
    'live_register',
    'live_attend',
    'live_questions',
    'live_ondemand',
    'conversion_academic',
    'conversion_article',
    'conversion_subscribe',
    'conversion_other',
    'conversion_print_archive',
    'conversion_0']

yvar = 'churn'

In [159]:
#checking class imbalance
print(df[yvar].value_counts())

1.0    19716
0.0    16419
Name: churn, dtype: int64


In [160]:
#checking to clean categorical vars
for i in xvars:
    print("\n")
    print(i)
    print(df[i].value_counts())



days_subscribed
31.000000      41
30.000000      21
1144.000000    18
61.000000      16
365.000000      9
               ..
171.947697      1
171.829699      1
171.771968      1
171.719375      1
314.776898      1
Name: days_subscribed, Length: 35920, dtype: int64


auto_renew
0.0    23771
1.0    12364
Name: auto_renew, dtype: int64


logins_last_30
0.0     28598
1.0      4887
2.0      1533
3.0       529
4.0       256
5.0       118
6.0        66
7.0        33
9.0        19
8.0        17
11.0       12
10.0       11
12.0        8
13.0        7
17.0        7
14.0        5
25.0        4
15.0        4
19.0        3
28.0        2
20.0        2
18.0        2
23.0        2
16.0        2
22.0        2
61.0        1
36.0        1
33.0        1
26.0        1
32.0        1
34.0        1
Name: logins_last_30, dtype: int64


pageviews_last_30
0.0      27281
1.0        903
2.0        743
3.0        642
4.0        563
5.0        435
6.0        377
7.0        303
8.0        291
9.0        289
10.0   

In [161]:
#checking to clean continuous vars
for i in xvars:
    print("\n")
    print(i)
    print(df[i].describe)



days_subscribed
<bound method NDFrame.describe of 0          -0.051470
1           0.000313
2           0.000509
3           0.000521
4           0.000544
            ...     
36211    1144.000000
36212    1144.000000
36213    1144.000000
36214    1144.000000
36215    1144.000000
Name: days_subscribed, Length: 36135, dtype: float64>


auto_renew
<bound method NDFrame.describe of 0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
36211    0.0
36212    0.0
36213    0.0
36214    0.0
36215    0.0
Name: auto_renew, Length: 36135, dtype: float64>


logins_last_30
<bound method NDFrame.describe of 0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
36211    0.0
36212    2.0
36213    0.0
36214    0.0
36215    0.0
Name: logins_last_30, Length: 36135, dtype: float64>


pageviews_last_30
<bound method NDFrame.describe of 0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
36211    0.0
36212    9.0
36213    0.0
36214 

In [162]:
#Rounding days_subscribed
df['days_subscribed'] = df['days_subscribed'].round(decimals=0)

In [181]:
#look to see if there are any missing values
pd.set_option('display.max_rows', 500)
df.isnull().sum()

Subscription ID                       0
Summary                               0
Create date                           0
start                                 0
End date- Ended                   16738
End date- Ongoing                     0
days_subscribed                       0
Subscription status                   0
churn                                 0
Upgrade status                        0
Trial status                          0
Trial period end date             36100
Trial price                       36101
Regular_price                         0
Auto-renew                            0
auto_renew                            0
Auto-renew disablement date       29428
Last active date                   9391
logins_last_30                        0
sessions_last_30                      0
pageviews_last_30                     0
Billing period                        0
Total charged                         0
Charge count                          0
Total refunded                        0


In [182]:
#look to see class imbalance
print(df['churn'].value_counts())

1.0    19716
0.0    16419
Name: churn, dtype: int64


In [165]:
#correlation matrix
corr_matrix = df[xvars].corr()
print(corr_matrix)

# TLDR: Sessions and Pageviews are highly correlated, logins also correlated decently.
# Sub Year correlated with Sub type: Digital and Premium
# Digital and Preimum are also highly correlated
# Live Variables all correlated highly

                   days_subscribed  auto_renew  logins_last_30  \
days_subscribed           1.000000   -0.309489        0.013679   
auto_renew               -0.309489    1.000000       -0.126156   
logins_last_30            0.013679   -0.126156        1.000000   
pageviews_last_30        -0.002501   -0.111210        0.539003   
promcode                 -0.096362   -0.047816        0.058786   
num_newsletters          -0.213383    0.051026       -0.000725   
login_app                -0.219548    0.110012       -0.019197   
sub_year                 -0.462190   -0.192054        0.136626   
sub_month                -0.066218   -0.032303        0.057020   
type_Digital              0.277104    0.159144       -0.089788   
type_Insider              0.054349   -0.109362        0.010569   
type_Premium             -0.290048   -0.101176        0.080872   
covid                    -0.013550    0.013342       -0.002206   
ukraine                  -0.144199    0.169231       -0.030624   
sessions_l

In [166]:
# Top 5 most correlated features with churn
corr_matrix_churn = df.corr()
print(corr_matrix_churn)

                          days_subscribed     churn  Trial price  \
days_subscribed                  1.000000 -0.363518    -0.116310   
churn                           -0.363518  1.000000     0.027736   
Trial price                     -0.116310  0.027736     1.000000   
Regular_price                    0.039866 -0.046218     0.993743   
auto_renew                      -0.309489  0.509713    -0.098085   
logins_last_30                   0.013679 -0.216057    -0.102131   
sessions_last_30                 0.007042 -0.200729    -0.099700   
pageviews_last_30               -0.002501 -0.188694    -0.091079   
Total charged                    0.101788 -0.049917     0.873110   
Charge count                     0.479305 -0.140381    -0.180311   
Total refunded                  -0.009658  0.019344     0.975283   
promcode                        -0.096362 -0.173124          NaN   
PSC subscriber number                 NaN       NaN          NaN   
Tax                             -0.061433 -0.283

Exploratory Tables and Figures

In [167]:
#Import the stuff from stata to here, remake the carts and tables

In [168]:
#App Login and Churn
churn_app = pd.crosstab(df['churn'], df['login_app'])
print(churn_app)
churn_app = churn_app.to_latex()

login_app      0     1
churn                 
0.0        14276  2143
1.0        15500  4216


/var/folders/3c/s8p5wvy902gfgnq64pms18vm0000gn/T/ipykernel_1497/2520400006.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  churn_app = churn_app.to_latex()


In [169]:
# Newsletters and Churn

churn_news = pd.crosstab(df['churn'], df['num_newsletters'])
print(churn_news)
churn_news = churn_news.to_latex()

num_newsletters      0     1    2    3    4    5    6   7    8
churn                                                         
0.0              11336  3589  553  314  252  128   64  68  115
1.0              12562  5079  729  412  347  225  113  95  154


/var/folders/3c/s8p5wvy902gfgnq64pms18vm0000gn/T/ipykernel_1497/1665162314.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  churn_news = churn_news.to_latex()


In [184]:
# Conversion and Churn

churn_convert = pd.crosstab(df['churn'], df['conversion'])
print(churn_convert)
churn_convert = churn_convert.to_latex()

conversion     0  academic  article  other  print_archive  subscribe
churn                                                               
0.0          761      2052     4634     33             12       8927
1.0         4249      2515     5805     37             17       7093


/var/folders/3c/s8p5wvy902gfgnq64pms18vm0000gn/T/ipykernel_1497/105433356.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  churn_convert = churn_convert.to_latex()


In [197]:
print("Average Sessions for ACTIVE: " + str(df.query('churn==0')['sessions_last_30'].describe()))
print("Average Sessions for CHURN: " + str(df.query('churn==1')['sessions_last_30'].describe()))

Average Sessions for ACTIVE: count    16419.000000
mean         4.422986
std         11.800307
min          0.000000
25%          0.000000
50%          0.000000
75%          4.000000
max        252.000000
Name: sessions_last_30, dtype: float64
Average Sessions for CHURN: count    19716.000000
mean         0.781903
std          5.246828
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        190.000000
Name: sessions_last_30, dtype: float64


In [198]:
print("Average Pageviews for ACTIVE: " + str(df.query('churn==0')['pageviews_last_30'].describe()))
print("Average Pageviews for CHURN: " + str(df.query('churn==1')['pageviews_last_30'].describe()))

Average Pageviews for ACTIVE: count    16419.000000
mean         9.662951
std         27.325186
min          0.000000
25%          0.000000
50%          0.000000
75%          7.000000
max        640.000000
Name: pageviews_last_30, dtype: float64
Average Pageviews for CHURN: count    19716.000000
mean         1.701309
std         12.584275
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        666.000000
Name: pageviews_last_30, dtype: float64


In [199]:
print("Average Logins for ACTIVE: " + str(df.query('churn==0')['logins_last_30'].describe()))
print("Average Logins for CHURN: " + str(df.query('churn==1')['logins_last_30'].describe()))

Average Logins for ACTIVE: count    16419.000000
mean         0.653085
std          1.519115
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         61.000000
Name: logins_last_30, dtype: float64
Average Logins for CHURN: count    19716.000000
mean         0.135727
std          0.749699
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         28.000000
Name: logins_last_30, dtype: float64


In [207]:
px.histogram(
    df,
    x="days_subscribed",
    facet_col="churn"
)

In [210]:
px.histogram(
    df.query("start < 2024"),
    x = "start", 
)

In [211]:
churn_group = pd.crosstab(df['churn'], df['group'])
print(churn_group)
churn_group = churn_group.to_latex()

group      0     1
churn             
0.0    16412     7
1.0    16199  3517


/var/folders/3c/s8p5wvy902gfgnq64pms18vm0000gn/T/ipykernel_1497/3196411407.py:3: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



Part 2: Feature Selection

In [171]:
#hypothesis testing/ p-values

In [172]:
#grid searching

In [173]:
#gravitational searching

In [174]:
#regularization

Part 3: Model Testing

In [175]:
#importing models

In [176]:
#hyperparameter optimization

In [177]:
#cross-validation

Part 4: Prediction Testing

In [178]:
#Confusion Matrix

In [179]:
#Precision, Recall, and Accuracy

In [180]:
# AUC Curve